![clothing_classification](clothing_classification.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [132]:
# Run the cells below first

In [133]:
!pip install torchmetrics
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [134]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [135]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [136]:
# Define the CNN model
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # 1 input channel for grayscale images
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
        )
        self.classifier = nn.Linear(64 * 7 * 7, num_classes)  # Adjust the dimensions

    def forward(self, x):  
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x

In [137]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Initialize the model, loss function, and optimizer
num_classes = 10  # FashionMNIST has 10 classes
model = Net(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 1  # Keeping epochs to 1 or 2 to keep the run time down
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")


Epoch 1, Loss: 0.4085


In [138]:
metric_accuracy = Accuracy(task="multiclass", num_classes=10)
metric_precision = Precision(task="multiclass", num_classes=10, average=None)
metric_recall = Recall(task="multiclass", num_classes=10, average=None)

model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predictions_ = torch.max(outputs, 1)
        predictions.extend(predictions_.cpu().tolist())
        true_labels.extend(labels.cpu().tolist())
        
        metric_accuracy(predictions_, labels)
        metric_precision(predictions_, labels)
        metric_recall(predictions_, labels)

accuracy = metric_accuracy.compute().item()
precision = metric_precision.compute()
recall = metric_recall.compute()

# Convert precision and recall to Python lists
precision = precision.tolist()
recall = recall.tolist()

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision_list}")
print(f"Recall: {recall_list}")



Accuracy: 0.8745999932289124
Precision: [0.8190476298332214, 0.9760956168174744, 0.8660714030265808, 0.9124087691307068, 0.7404958605766296, 0.9497584700584412, 0.7288329601287842, 0.9671772718429565, 0.972000002861023, 0.9196597337722778]
Recall: [0.8600000143051147, 0.9800000190734863, 0.7760000228881836, 0.875, 0.8960000276565552, 0.9829999804496765, 0.6370000243186951, 0.8840000033378601, 0.972000002861023, 0.9729999899864197]
